In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import time

from datetime import datetime
from dateutil import parser

import pandas as pd

## NBCNews

In [2]:
options = Options()
#options.headless = True
b = webdriver.Chrome(options=options)

url = 'https://www.nbcnews.com/search/?q=gaza+war'
b.get(url)

In [3]:
dropdown = b.find_element(By.CLASS_NAME, 'gsc-selected-option-container')
dropdown.click()

time.sleep(1)

date_option = b.find_element(By.XPATH, '/html/body/div[@id="content"]/div[@id="___gcse_0"]/div[@class="gsc-control-cse gsc-control-cse-en"]/div[@class="gsc-control-wrapper-cse"]/div[@class="gsc-results-wrapper-nooverlay gsc-results-wrapper-visible"]/div[@class="gsc-above-wrapper-area"]/table/tbody/tr/td/div/div/div[@class="gsc-option-menu"]/div[@class="gsc-option-menu-item"][1]')
date_option.click()

In [4]:
response_text = b.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[2]/div').get_attribute("outerHTML")

soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('div', class_='gsc-cursor-page')
last_page_number = int(div_list[-1].text)
last_page_number

10

In [12]:
def convert_to_local_time(soup_find_string):
    
    time_string = soup_find_string.split("...")[0].rstrip()
    time_strings_list = time_string.split()
    duration_str = time_strings_list[1]
    
    if (duration_str == 'second') or (duration_str == 'seconds'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(seconds=delta_num)
    elif (duration_str == 'minute') or (duration_str == 'minutes'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(minutes=delta_num)
    elif (duration_str == 'hour') or (duration_str == 'hours'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(hours=delta_num)
    elif (duration_str == 'day') or (duration_str == 'days'):
        delta_num = int(time_strings_list[0])
        time_stamp = datetime.now() - timedelta(days=delta_num)
    else:
        date_format = '%b %d, %Y'
        time_stamp = datetime.strptime(time_string, date_format)
    
    return time_stamp

In [6]:
next_page_number = 1

while next_page_number <= last_page_number:
    
    # get the page number of the next page
    current_page_str = b.find_element(By.CLASS_NAME, 'gsc-cursor-current-page').text
    next_page_number = int(current_page_str) + 1
    next_page_number
    
    if next_page_number > last_page_number:
        break

    # click the next page link
    next_page_button = b.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[{}]'.format(next_page_number))
    next_page_button.click()
    time.sleep(1)

In [13]:
response_text = b.find_element(By.CLASS_NAME, 'gsc-expansionArea').get_attribute("outerHTML")

soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('div', class_='gs-webResult gs-result')

# for result in div_list:
#     article_url = result.find('a')['href']
#     print("Article URL:", article_url)

oldes_time_string = div_list[-1].find(class_='gs-bidi-start-align gs-snippet').text

oldest_article_time = convert_to_local_time(oldes_time_string)

In [5]:
while oldest_article_time > datetime(2023, 10, 1, 0, 0):
    load_more_button.click()
    time.sleep(1)
    
    response_text = b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

    soup = BeautifulSoup(response_text, 'html.parser')

    # Find the specific elements that contain the article links
    div_list = soup.find_all('div', class_='search-result-indiv')

    # for result in div_list:
    #     article_url = result.find('a')['href']
    #     print("Article URL:", article_url)

    oldest_time_string = div_list[-1].find('h5').text
    oldest_article_time = parser.parse(oldest_time_string)

In [6]:
# Finally, you can retrieve the loaded content
page_source = b.page_source

# Close the browser
b.quit()

In [9]:
# save the following info to a dataframe

date_list = []
url_list = []
title_list = []

for result in div_list:
    
    article_date_string = result.find('h5').text
    article_date = parser.parse(article_date_string)
    date_list.append(article_date)
    
    article_url = result.find('a')['href']
    url_list.append(article_url)
    
    article_title = result.find('h3', class_='search-result-title').text
    title_list.append(article_title)
    
#     print(article_date)
#     print(article_url)
#     print(article_title)

d = {'datetime': date_list, 'url': url_list, 'title':title_list}
df_reuters_articles = pd.DataFrame(data=d)

/Users/zhounanlin/anaconda3/envs/anlp/lib/python3.11/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [10]:
df_reuters_articles

,datetime,url,title
0,2023-10-30 00:49:00,/article/idUSL1N3C009L,China stocks rise on support signs; HK dips on...
1,2023-10-30 00:25:00,/article/idUSKBN31U016,"Oil down 1% ahead of U.S., China data"
2,2023-10-30 00:25:00,/article/idUSL1N3C007N,"UPDATE 1-Oil down 1% ahead of U.S., China data"
3,2023-10-30 00:11:00,/article/idUSL4N3C00U4,Australia dollar gains on stronger retail data...
4,2023-10-29 23:34:00,/article/idUSKBN31T0FZ,Anti-Israeli protesters storm Russia's Dagesta...
...,...,...,...
985,2023-10-04 08:34:00,/article/idUSKBN31410C,Gaza unrest sends message about economic miser...
986,2023-09-27 11:01:00,/article/idUSKBN30X1FE,"UN peace envoy, Egypt working to restore calm ..."
987,2023-09-24 06:14:00,/article/idUSL1N3B001W,UPDATE 2-Israeli forces kill two Palestinians ...
988,2023-09-22 14:54:00,/article/idUSKBN30S1KD,Israeli drone hits Gaza as violent protests rage


## CNN

## Reuters

In [2]:
options = Options()
#options.headless = True
b = webdriver.Chrome(options=options)

url = 'https://www.reuters.com/search/news?blob=gaza+war&sortBy=date&dateRange=all'
b.get(url)

load_more_button = b.find_element(By.CLASS_NAME, 'search-result-more-txt')

In [3]:
# b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

In [4]:
response_text = b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

soup = BeautifulSoup(response_text, 'html.parser')

# Find the specific elements that contain the article links
div_list = soup.find_all('div', class_='search-result-indiv')

# for result in div_list:
#     article_url = result.find('a')['href']
#     print("Article URL:", article_url)

oldes_time_string = div_list[-1].find('h5').text
oldest_article_time = parser.parse(oldes_time_string)

/Users/zhounanlin/anaconda3/envs/anlp/lib/python3.11/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [5]:
while oldest_article_time > datetime(2023, 10, 1, 0, 0):
    load_more_button.click()
    time.sleep(1)
    
    response_text = b.find_element(By.CLASS_NAME, 'search-result-list').get_attribute("outerHTML")

    soup = BeautifulSoup(response_text, 'html.parser')

    # Find the specific elements that contain the article links
    div_list = soup.find_all('div', class_='search-result-indiv')

    # for result in div_list:
    #     article_url = result.find('a')['href']
    #     print("Article URL:", article_url)

    oldest_time_string = div_list[-1].find('h5').text
    oldest_article_time = parser.parse(oldest_time_string)

In [6]:
# Finally, you can retrieve the loaded content
page_source = b.page_source

# Close the browser
b.quit()

In [9]:
# save the following info to a dataframe

date_list = []
url_list = []
title_list = []

for result in div_list:
    
    article_date_string = result.find('h5').text
    article_date = parser.parse(article_date_string)
    date_list.append(article_date)
    
    article_url = result.find('a')['href']
    url_list.append(article_url)
    
    article_title = result.find('h3', class_='search-result-title').text
    title_list.append(article_title)
    
#     print(article_date)
#     print(article_url)
#     print(article_title)

d = {'datetime': date_list, 'url': url_list, 'title':title_list}
df_reuters_articles = pd.DataFrame(data=d)

/Users/zhounanlin/anaconda3/envs/anlp/lib/python3.11/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [10]:
df_reuters_articles

,datetime,url,title
0,2023-10-30 00:49:00,/article/idUSL1N3C009L,China stocks rise on support signs; HK dips on...
1,2023-10-30 00:25:00,/article/idUSKBN31U016,"Oil down 1% ahead of U.S., China data"
2,2023-10-30 00:25:00,/article/idUSL1N3C007N,"UPDATE 1-Oil down 1% ahead of U.S., China data"
3,2023-10-30 00:11:00,/article/idUSL4N3C00U4,Australia dollar gains on stronger retail data...
4,2023-10-29 23:34:00,/article/idUSKBN31T0FZ,Anti-Israeli protesters storm Russia's Dagesta...
...,...,...,...
985,2023-10-04 08:34:00,/article/idUSKBN31410C,Gaza unrest sends message about economic miser...
986,2023-09-27 11:01:00,/article/idUSKBN30X1FE,"UN peace envoy, Egypt working to restore calm ..."
987,2023-09-24 06:14:00,/article/idUSL1N3B001W,UPDATE 2-Israeli forces kill two Palestinians ...
988,2023-09-22 14:54:00,/article/idUSKBN30S1KD,Israeli drone hits Gaza as violent protests rage
